In [1]:
%load_ext autoreload
%autoreload 2

# Import library

In [13]:
import numpy as np
import math
# from . import calc_util as cu
import sys
sys.path.append('src')
import enex_analysis as enex
import dartwork_mpl as dm
import matplotlib.pyplot as plt
import CoolProp.CoolProp as CP
import numpy as np
from tqdm import tqdm
import pandas as pd
import matplotlib.ticker as ticker
dm.use_dmpl_style()


# 0. 준비

In [14]:
## Fontsize 지정
plt.rcParams['font.size'] = 9

fs = {
    'label': dm.fs(0),
    'tick': dm.fs(-1.5),
    'legend': dm.fs(-2.0),
    'subtitle': dm.fs(-0.5),
    'cbar_tick': dm.fs(-2.0),
    'cbar_label': dm.fs(-2.0),
    'cbar_title': dm.fs(-1),
    'setpoint': dm.fs(-1),
    'text': dm.fs(-3.0),
            }

pad = {
    'label': 6,
    'tick': 5,
}

LW = np.arange(0.25, 3.0, 0.25)

# 1. 단일 운전점 최적화 & 사이클 도식

In [15]:
# =============================================================================
# 1) 단일 운전점 최적화 & 사이클 도식
# =============================================================================
GSHPB = enex.GroundSourceHeatPumpBoiler2(
    refrigerant  = 'R410A',
    V_disp_cmp   = 0.00005,   #0.00001 ~ 0.00011m³/rev
    eta_cmp_isen = 0.7,
    eta_cmp_vol  = 0.85,
    
    T_b_f_in     = 15.0,
    Ts           = 15.0,
    
    UA_cond      = 500,       # W/K
    UA_evap      = 500,       # W/K
    
    D_b          = 0,         # Borehole depth [m]
    H_b          = 200,       # Borehole height [m]
    r_b          = 0.08,      # Borehole radius [m]
    R_b          = 0.108,     # Effective borehole thermal resistance [mK/W]
    
    dV_b_f       = 24,        # Volumetric flow rate of fluid [L/min]
    
    k_s          = 2.0,
    c_s          = 800,
    rho_s        = 2000,
    
    E_pmp        = 200,
)

ref_loop_result = GSHPB._find_ref_loop_optimal_operation(T_tank_w=60, Q_cond_load=-8000)

print(
    f'Temperature ' + '='*10 + '\n',
    f'T1: {enex.K2C(ref_loop_result["T1"]):.2f} °C,\n'
    f'T2: {enex.K2C(ref_loop_result["T2"]):.2f} °C,\n'
    f'T3: {enex.K2C(ref_loop_result["T3"]):.2f} °C,\n'
    f'T4: {enex.K2C(ref_loop_result["T4"]):.2f} °C,\n'
    f'\nPressure ' + '='*10 + '\n',
    f'P1: {ref_loop_result["P1"]*enex.Pa2kPa:.2f} kPa,\n'
    f'P2: {ref_loop_result["P2"]*enex.Pa2kPa:.2f} kPa,\n'
    f'P3: {ref_loop_result["P3"]*enex.Pa2kPa:.2f} kPa,\n'
    f'P4: {ref_loop_result["P4"]*enex.Pa2kPa:.2f} kPa,\n'
    f'\nPerformance ' + '='*10 + '\n',
    f'E_cmp: {ref_loop_result["E_cmp"]:.1f} W,\n'
    f'cmp_rps: {ref_loop_result["cmp_rps"]:.2f} rps,\n'
    f'cmp_rpm: {ref_loop_result["cmp_rps"]*60:.0f} rpm,\n'
)

GSHPB.plot_cycle_diagrams(result=ref_loop_result, save_path='figure/HeatPump_model/HeatPump_cycle_diagram.png')

# State 글자 너무 큼
# R410 히트펌프 사이클 돌릴 때, 압력이 어느수준에서 실제로 노는지 확인해봐야할듯 (모리엘선도 P-h)
# Pressure - 500 정도에서 lim 잘라서 보는게 좋을듯

'''
1. 저압측 (증발기) 압력 ($P_{low}$): 995.07 kPa이 압력에서 R410A의 포화 온도(끓는점)는 약 7°C 입니다.
T4: 7.06 °C (팽창밸브 출구)분석: 팽창밸브를 통과한 냉매는 액체와 기체가 섞인 2상(two-phase) 상태가 됩니다. 이때의 온도는 포화 온도와 일치해야 합니다.
7.06°C는 포화 온도 7°C와 거의 일치하므로 매우 현실적인 값입니다
T1: 7.11 °C (압축기 입구)분석: 냉매는 증발기를 거치며 모두 기화된 후, 약간 과열되어(superheated) 압축기로 들어갑니다.
과열도(Superheat) = $T_1$ - $T_{sat,low}$ = 7.11°C - 7°C ≈ 0.11°C이 값은 과열도가 매우 낮은 상태지만, 모델링 상으로는 열역학적으로 올바른 상태 (과열 증기)를 나타냅니다.

2. 고압측 (응축기) 압력 ($P_{high}$): 4123.88 kPa이 압력에서 R410A의 포화 온도(응축 온도)는 약 63.7°C 입니다.
T2: 97.50 °C (압축기 출구)분석: 압축기를 나온 냉매는 고온·고압의 과열 증기 상태여야 합니다.
$T_2$ (97.50°C)는 포화 온도(63.7°C)보다 훨씬 높으므로, 이는 정상적인 과열 상태입니다.
압축기 토출 과열도(Discharge Superheat) = $T_2$ - $T_{sat,high}$ = 97.50°C - 63.7°C ≈ 33.8°C이 값은 압축기 성능을 고려할 때 매우 합리적인 수준입니다.
T3: 63.25 °C (응축기 출구)분석: 응축기를 통과한 냉매는 모두 액화된 후, 팽창밸브로 들어가기 전에 약간 과냉각(subcooled) 됩니다.
과냉도(Subcooling) = $T_{sat,high}$ - $T_3$ = 63.7°C - 63.25°C ≈ 0.45°C이 값은 과냉도가 매우 작은 상태지만, 열역학적으로 올바른 상태 (과냉각 액체)를 나타냅니다.
'''

Temperature ==========
 T1: 6.20 °C,
T2: 99.83 °C,
T3: 64.33 °C,
T4: 6.15 °C,

Pressure ==========
 P1: 968.03 kPa,
P2: 4221.46 kPa,
P3: 4221.46 kPa,
P4: 968.03 kPa,

Performance ==========
 E_cmp: 2859.3 W,
cmp_rps: 31.62 rps,
cmp_rpm: 1897 rpm,



'\n1. 저압측 (증발기) 압력 ($P_{low}$): 995.07 kPa이 압력에서 R410A의 포화 온도(끓는점)는 약 7°C 입니다.\nT4: 7.06 °C (팽창밸브 출구)분석: 팽창밸브를 통과한 냉매는 액체와 기체가 섞인 2상(two-phase) 상태가 됩니다. 이때의 온도는 포화 온도와 일치해야 합니다.\n7.06°C는 포화 온도 7°C와 거의 일치하므로 매우 현실적인 값입니다\nT1: 7.11 °C (압축기 입구)분석: 냉매는 증발기를 거치며 모두 기화된 후, 약간 과열되어(superheated) 압축기로 들어갑니다.\n과열도(Superheat) = $T_1$ - $T_{sat,low}$ = 7.11°C - 7°C ≈ 0.11°C이 값은 과열도가 매우 낮은 상태지만, 모델링 상으로는 열역학적으로 올바른 상태 (과열 증기)를 나타냅니다.\n\n2. 고압측 (응축기) 압력 ($P_{high}$): 4123.88 kPa이 압력에서 R410A의 포화 온도(응축 온도)는 약 63.7°C 입니다.\nT2: 97.50 °C (압축기 출구)분석: 압축기를 나온 냉매는 고온·고압의 과열 증기 상태여야 합니다.\n$T_2$ (97.50°C)는 포화 온도(63.7°C)보다 훨씬 높으므로, 이는 정상적인 과열 상태입니다.\n압축기 토출 과열도(Discharge Superheat) = $T_2$ - $T_{sat,high}$ = 97.50°C - 63.7°C ≈ 33.8°C이 값은 압축기 성능을 고려할 때 매우 합리적인 수준입니다.\nT3: 63.25 °C (응축기 출구)분석: 응축기를 통과한 냉매는 모두 액화된 후, 팽창밸브로 들어가기 전에 약간 과냉각(subcooled) 됩니다.\n과냉도(Subcooling) = $T_{sat,high}$ - $T_3$ = 63.7°C - 63.25°C ≈ 0.45°C이 값은 과냉도가 매우 작은 상태지만, 열역학적으로 올바른 상태 (과냉각 액체)를 나타냅니다.\n'

## 1.1 Constant

In [16]:
c_w = 4186  # J/kgK
rho_w = 1000  # kg/m3

# 2. Function

In [52]:
# =============================================================================
# 4. 시뮬레이션 실행 함수 및 로직 (개선된 버전)
# =============================================================================
def run_simulation(gshpb, config, serv_sched):
    ''' 
    Args:
        gshpb: GroundSourceHeatPumpBoiler2 객체
        config: 시뮬레이션 설정 딕셔너리
        serv_sched: 급탕 부하 스케줄 (배열)
    Returns:
        results_df: 시뮬레이션 결과가 담긴 DataFrame
    '''
    # --- 초기화 ---
    time = config['time']
    tN = len(time)
    results_df = pd.DataFrame(index=range(tN), columns=config['results_keys'])
    
    # 동적 상태 변수 초기화
    T_tank_w_series = np.full(tN, np.nan)
    T_tank_w_series[0] = config['T_tank_w_init']
    Q_b_unit_pulse = np.zeros(tN)
    Q_b_unit_old = 0
    is_on_prev_step = False

    # --- 시뮬레이션 루프 ---
    for n in tqdm(range(tN), desc="Simulating"):
        # ⭐️ 1. 매 스텝마다 결과를 담을 '결과 수집기' 딕셔너리 생성
        step_results = {}
        
        # 현재 값들을 수집기에 추가
        T_tank_w_n = T_tank_w_series[n]
        step_results['T_tank_w'] = T_tank_w_n

        # 2. 제어 상태 결정 (수집기 전달)
        is_on_n, Q_cond_load_n = update_control_state(
            T_tank_w_n, is_on_prev_step, serv_sched[n], config, step_results
        )
        
        # 3. 히트펌프 운전점 계산
        ref_loop_result = gshpb._find_ref_loop_optimal_operation(T_tank_w=T_tank_w_n, Q_cond_load=Q_cond_load_n)
        if ref_loop_result is None:
            ref_loop_result = gshpb._off_result(T_tank_w_n)
            is_on_n = False
        
        # 계산 결과를 수집기에 추가
        step_results.update(ref_loop_result)
        step_results['is_on'] = is_on_n

        # 4. 지중 온도 업데이트 (수집기 전달)
        Q_b_unit_pulse, Q_b_unit_old = update_ground_temperature(
            n, ref_loop_result, Q_b_unit_pulse, Q_b_unit_old, gshpb, config, step_results
        )
        # 5. 다음 스텝 탱크 온도 계산
        if n < tN - 1:
            Q_tank_in = -ref_loop_result.get('Q_ref_cond', 0.0)
            Q_net = Q_tank_in - step_results['total_loss'] # 수집기에서 값 사용
            T_tank_w_series[n+1] = T_tank_w_n + (Q_net / config['C_tank']) * config['dt']

        gshpb.T_b_f_in_K = step_results['T_b_f_in'] # 수집기에서 값 사용  
        is_on_prev_step = is_on_n
        
        # ⭐️ 6. 매우 깔끔해진 결과 저장 단계
        store_results(results_df, n, step_results)
        
    return results_df


# ⭐️ 함수 시그니처와 내용 변경
def update_control_state(T_tank_w_n, is_on_prev, schedule_n, config, results_container):
    """결과를 results_container에 직접 저장하고, 계산에 필요한 값만 반환합니다."""
    # ... (손실 계산 로직은 동일) ...
    Q_env_loss = config['UA_tank'] * (T_tank_w_n - config['T0'])
    den = max(1e-6, enex.C2K(T_tank_w_n) - enex.C2K(config['T_w_sup']))
    alp = min(1.0, max(0.0, enex.C2K(config['T_serv_w']) - enex.C2K(config['T_w_sup'])) / den)
    dV_w_serv = schedule_n * config['dV_w_serv_m3s']
    dV_w_sup_tank = alp * dV_w_serv
    dV_w_sup_mix = (1 - alp) * dV_w_serv    
    Q_use_loss = schedule_n * (c_w * rho_w * dV_w_sup_tank * (T_tank_w_n - config['T_w_sup']))
    total_loss = Q_env_loss + Q_use_loss
    
    # On/Off 결정
    if T_tank_w_n < config['T_tank_w_lower_bound']: is_on = True
    elif T_tank_w_n > config['T_tank_w_setpoint']: is_on = False
    else: is_on = is_on_prev
    Q_cond_load_n = -config['heater_capacity'] if is_on else 0.0
    
    # ⭐️ 계산된 결과를 딕셔너리에 직접 추가
    results_container['dV_w_sup_tank'] = dV_w_sup_tank
    results_container['dV_w_serv'] = dV_w_serv
    results_container['dV_w_sup_mix'] = dV_w_sup_mix
    results_container['T_tank_w'] = T_tank_w_n
    results_container['is_on'] = is_on
    results_container['dV_w_serv'] = dV_w_serv
    results_container['Q_env_loss'] = Q_env_loss
    results_container['Q_use_loss'] = Q_use_loss
    results_container['total_loss'] = total_loss
    
    # ⭐️ 다음 '계산'에 필요한 값만 반환
    return is_on, Q_cond_load_n

# ⭐️ 함수 시그니처와 내용 변경
def update_ground_temperature(n, ref_res, Q_b_pulse, Q_b_unit_old, gshpb, config, results_container):
    """결과를 results_container에 직접 저장하고, 상태 업데이트에 필요한 값만 반환합니다."""
    # ... (지중 온도 계산 로직은 동일) ...
    Q_b_unit = (ref_res.get('Q_ref_evap', 0.0) - gshpb.E_pmp) / gshpb.H_b if ref_res.get('is_on') else 0.0
    
    if abs(Q_b_unit - Q_b_unit_old) > 1e-6:
        Q_b_pulse[n] = Q_b_unit - Q_b_unit_old
        Q_b_unit_old_new = Q_b_unit
    else:
        Q_b_unit_old_new = Q_b_unit_old

    # ... (g-function 계산 등 나머지 로직) ...
    Q_b_unit = (ref_res.get('Q_ref_evap', 0.0) - gshpb.E_pmp) / gshpb.H_b if ref_res.get('is_on') else 0.0
    
    if abs(Q_b_unit - Q_b_unit_old) > 1e-6: # 만약 Q_b이 이전 스텝과 일정 수준 이상 차이가 난다면 펄스가 나타난 것으로 간주
        Q_b_pulse[n] = Q_b_unit - Q_b_unit_old # 펄스는 이전 값과의 차이
        Q_b_unit_old = Q_b_unit # 업데이트
    
    pulses_idx = np.flatnonzero(Q_b_pulse[:n+1])
    dQ = Q_b_pulse[pulses_idx]
    tau = config['time'][n] - config['time'][pulses_idx]
    
    # g-function 계산은 여전히 루프가 필요
    g_n = np.array([enex.G_FLS(t, gshpb.k_s, gshpb.alp_s, gshpb.r_b, gshpb.H_b) for t in tau])
    g_conv = np.sum(g_n)
    
    dT_b = np.dot(dQ, g_n)
    T_b = gshpb.Ts - dT_b
    
    T_f_bh = T_b - Q_b_unit * gshpb.R_b
    Q_g_total = Q_b_unit * gshpb.H_b
    T_b_f_in = T_f_bh - Q_g_total / (2 * c_w * rho_w * gshpb.dV_b_f)
    T_b_f_out = T_f_bh + Q_g_total / (2 * c_w * rho_w * gshpb.dV_b_f)
    
    T0_K = enex.C2K(config['T0'])
    Xg = (1 - T0_K / gshpb.Ts) * Q_g_total
    Xb = (1 - T0_K / T_b) * Q_g_total
    Xc_g = Xg - Xb
    
    # ⭐️ 계산된 결과를 딕셔너리에 직접 추가
    results_container['T_b'] = T_b
    results_container['T_b_f_in'] = T_b_f_in
    results_container['T_b_f_out'] = T_b_f_out
    results_container['Q_b_unit'] = Q_b_unit
    results_container['g_conv'] = g_conv
    results_container['Xc_g'] = Xc_g
    results_container['Xg'] = Xg
    results_container['Xb'] = Xb
    results_container['dQ'] = Q_b_pulse # dQ는 배열이지만, 필요하다면 n번째 값을 저장
    
    T_tank_w_n = results_container['T_tank_w']
    
    E_cmp = results_container.get('E_cmp', 0.0)
    Q_ref_cond = results_container.get('Q_ref_cond', 0.0)
    T3_K = results_container.get('T3', enex.C2K(T_tank_w_n))
    
    results_container['X_cmp'] = E_cmp
    results_container['X_pmp'] = gshpb.E_pmp if results_container.get('is_on') else 0.0
    results_container['X_ref_tank'] = Q_ref_cond * (1 - T0_K / T3_K)
    results_container['COP'] = abs(Q_ref_cond) / (E_cmp + gshpb.E_pmp) if (E_cmp + gshpb.E_pmp) > 0 else 0.0
    
    # ⭐️ 다음 루프에서 상태 유지를 위해 필요한 값만 반환
    return Q_b_pulse, Q_b_unit_old_new,


# ⭐️ 완전히 새로워지고 단순해진 함수
def store_results(df, n, results_container, ):
    """
    '결과 수집기' 딕셔너리를 받아 DataFrame의 n번째 행에 저장합니다.
    엑서지처럼 여기서 추가 계산이 필요한 항목들도 처리합니다.
    """
    # 2. 수집기의 모든 값을 DataFrame에 한 번에 저장
    for key, value in results_container.items():
        # df의 columns에 해당 key가 있을 경우에만 저장 (안전장치)
        if key in df.columns:
            df.loc[n, key] = value

# 3. 시뮬레이션 설정

In [53]:
# =============================================================================
# 5. 메인 실행 블록
# =============================================================================
# --- 1. 시뮬레이션 설정 (중앙 관리) ---
config = {
        'simulation_period_sec': 240 * enex.h2s,
        'heater_capacity': 8000.0, # W
        'dt': 2 * enex.m2s, # 2분
        'T_tank_w_setpoint': 65.0,
        'T_tank_w_lower_bound': 55.0,
        'T_tank_w_init': 60.0,
        'T0': 0.0,
        'T_serv_w': 40.0,
        'T_w_sup': 15.0,
        'dV_w_serv_m3s': 5 * enex.L2m3 / enex.m2s,
        'results_keys': [
            'T_b', 'T_b_f_in', 'T_b_f_out', 'T_tank_w',
            'COP', 'g_conv',
            'Q_b_unit',
            'Q_ref_cond', 'Q_ref_evap',
            'Q_env_loss', 'Q_use_loss', 'total_loss',
            'Q_ref',
            'is_on',
            'E_cmp', 'E_pmp',
            'Xc_g', 'Xg', 'Xb',
            'X_ref_tank',
            'dV_w_serv',
            'dV_w_sup_tank', 'dV_w_sup_mix',
            # 필요한 다른 결과 키들 추가...
        ]
    }
# --- 2. 시간 및 부하 프로파일 생성 ---
config['time'] = np.arange(0, config['simulation_period_sec'], config['dt'])
tN = len(config['time'])

serv_sched_hourly = [0]*6 + [1]*1 + [0]*11 + [1]*1 + [0]*5 # 6-7h, 18-19h
serv_sched_minutely = np.repeat(serv_sched_hourly, int(enex.h2s / config['dt']))
num_days = int(np.ceil(tN / len(serv_sched_minutely)))
serv_sched = np.tile(serv_sched_minutely, num_days)[:tN]

# --- 3. 모델 및 기타 파라미터 초기화 ---
gshpb_model = enex.GroundSourceHeatPumpBoiler2(
    V_disp_cmp=0.00005, eta_cmp_isen=0.7, T_b_f_in=16.0, Ts=16.0
)

tank_params = {'r0': 0.2, 'H': 0.8}
UA_tank = enex.calc_simple_tank_UA(
    r0=tank_params['r0'], H=tank_params['H']
)
config['UA_tank'] = UA_tank
config['C_tank'] = c_w * rho_w * (math.pi * tank_params['r0']**2 * tank_params['H'])
config['T0_K'] =enex.C2K(config['T0'])



# 4. Simulation 실행 및 csv 저장

In [55]:
results_dataframe = run_simulation(gshpb_model, config, serv_sched)
results_dataframe.to_csv('gshpb_simulation_results_refactored.csv')

Simulating: 100%|██████████| 7200/7200 [00:24<00:00, 297.27it/s]


# 4. Visualization

## 4.1 Data import

In [56]:
df = pd.read_csv('gshpb_simulation_results_refactored.csv', index_col=0)

## 4.2 Plot code

In [63]:
# =============================================================================
# 5) (옵션) 플롯 시작부
#    - 한글 폰트 설정 등은 사용 OS 환경에 맞게 수동 설정
# =============================================================================
def plot_simple_graph(df_column, time, xlabel, ylabel, xmin = 0, xmax=24, Kelvin=False, color = 'dm.blue5', savepath = None):
    # 온도 데이터를 섭씨(°C)로 변환하여 플로팅
    if Kelvin:
        y = enex.K2C(df_column)
    else:
        y = df_column
    x = time * enex.s2h  # 초를 시간으로 변환

    fig, ax = plt.subplots(figsize=(dm.cm2in(16), dm.cm2in(6)))
    
    # --- 마이너 틱 설정 ---

    # X축: 주 틱 사이를 5개의 간격으로 나눔 (마이너 틱 4개 생성)
    ax.xaxis.set_minor_locator(ticker.AutoMinorLocator(2))

    # Y축: 주 틱 사이를 2개의 간격으로 나눔 (마이너 틱 1개 생성)
    ax.yaxis.set_minor_locator(ticker.AutoMinorLocator(2))

    ax.plot(x, y, color=color, linewidth=0.5,)
    ax.set_xlim(xmin, xmax)
    if xmax % 24 == 0: 
        if xmax == 24:
            ax.set_xticks(np.arange(0, 25, 2))
        elif xmax == 48:
            ax.set_xticks(np.arange(0, xmax+1, 6))
        elif xmax == 72:
            ax.set_xticks(np.arange(0, xmax+1, 12))
        elif xmax > 72:
            ax.set_xticks(np.arange(0, xmax+1, 24))

    ax.tick_params(axis='both', which='both', labelsize=fs['tick'], pad=pad['tick'])
    ax.set_ylim(np.nanmin(y)*0.9, np.nanmax(y)*1.1)
    # ax.set_ylim(50, 55)
    ax.set_xlabel(xlabel, fontsize=fs['label'], labelpad=pad['label'])
    ax.set_ylabel(ylabel, fontsize=fs['label'], labelpad=pad['label'])
    dm.simple_layout(fig, margins=(0.05, 0.05, 0.05, 0.05), bbox = [0, 1, 0.02, 1])
    plt.savefig(f'{savepath}.png')
    dm.save_and_show(fig,)
    plt.close()

def plot_multi_graph(df_columns, legends, time, xlabel, ylabel, linestyles=None, colors=None, xmin=0, xmax=24, Kelvin=False, savepath=None):
    """
    여러 개의 데이터 열을 받아 하나의 그래프에 플로팅하는 함수.

    Args:
        *df_columns: 플로팅할 데이터 열들 (예: df['col1'], df['col2'], ...)
        labels (list or tuple): 각 데이터 열에 해당하는 레이블 리스트
        time (array-like): x축 시간 데이터
        ... (기존 인자들과 동일)
        colors (list or tuple, optional): 각 플롯에 사용할 색상 리스트. 기본값 None.
    """
    if len(df_columns) != len(legends):
        raise ValueError("데이터 컬럼의 개수와 레이블의 개수가 일치해야 합니다.")

    # --- 1. 그래프 기본 설정 ---
    fig, ax = plt.subplots(figsize=(dm.cm2in(16), dm.cm2in(6)))
    x = time * enex.s2h  # 초를 시간으로 변환
    
    # 기본 색상 리스트 (colors 인자가 주어지지 않을 경우 사용)
    if colors is None:
        colors = ['dm.blue5', 'dm.orange5', 'dm.green5', 'dm.red5', 'dm.violet5',
                  'dm.brown5', 'dm.pink5', 'dm.gray5', 'dm.yellow5', 'dm.cyan5']
    
    # --- 2. 모든 데이터를 바탕으로 Y축 min/max 계산 ---
    global_min = np.inf
    global_max = -np.inf
    
    processed_ys = []
    for col in df_columns:
        y = enex.K2C(col) if Kelvin else col
        processed_ys.append(y)
        if not y.empty:
            global_min = min(global_min, np.nanmin(y))
            global_max = max(global_max, np.nanmax(y))
            
    # --- 3. 반복문을 통해 여러 데이터 플로팅 ---
    for i, y_data in enumerate(processed_ys):
        ax.plot(x, y_data, 
                color=colors[i % len(colors)],  # 색상 순환 사용
                linewidth=0.8, 
                label=legends[i],
                linestyle=linestyles[i % len(linestyles)] if linestyles is not None else ['-', '--', '-.', ':'][i % 4],
                )               # 범례를 위한 레이블 추가

    # --- 4. 축 및 레이아웃 설정 ---
    ax.set_xlim(xmin, xmax)
    if xmax % 24 == 0: 
        if xmax == 24:
            ax.set_xticks(np.arange(0, 25, 2))
        elif xmax == 48:
            ax.set_xticks(np.arange(0, xmax+1, 6))
        elif xmax == 72:
            ax.set_xticks(np.arange(0, xmax+1, 12))
        elif xmax > 72:
            ax.set_xticks(np.arange(0, xmax+1, 24))
    
    ax.xaxis.set_minor_locator(ticker.AutoMinorLocator(2))
    ax.yaxis.set_minor_locator(ticker.AutoMinorLocator(2))
    
    ax.tick_params(axis='both', which='both', labelsize=fs['tick'], pad=pad['tick'])
    ax.set_ylim(global_min * 0.95, global_max * 1.2) # 전체 데이터 기준 ylim 설정
    
    ax.set_xlabel(xlabel, fontsize=fs['label'], labelpad=pad['label'])
    ax.set_ylabel(ylabel, fontsize=fs['label'], labelpad=pad['label'])
    
    ax.legend(ncol = 4) # 범례 표시
    dm.simple_layout(fig, margins=(0.05, 0.05, 0.05, 0.05), bbox=[0, 1, 0.02, 1])
    
    if savepath:
        plt.savefig(f'{savepath}.png')
        
    dm.save_and_show(fig)
    plt.close()

In [70]:
folder_path = 'figure/HeatPump_model'

xmax = 240

# plot_simple_graph(df['is_on'], config['time'],'Elapsed time [h]', 'Heater on/off [ - ]', xmax = xmax, color='dm.green5', savepath = f'{folder_path}/is_on_{xmax}')
# plot_simple_graph(df['dV_w_serv']*enex.m32L/enex.s2m, config['time'],'Elapsed time [h]', 'Water use schedule (L/min)', xmax = xmax, color='dm.violet5', savepath = f'{folder_path}/dV_w_serv_{xmax}')
# plot_simple_graph(df['dV_w_sup_tank']*enex.m32L/enex.s2m, config['time'],'Elapsed time [h]', 'Water supply to tank (L/min)', xmax = xmax, color='dm.violet5', savepath = f'{folder_path}/dV_w_sup_tank_{xmax}')
# plot_multi_graph((df['dV_w_serv']*enex.m32L/enex.s2m, df['dV_w_sup_tank']*enex.m32L/enex.s2m, df['dV_w_sup_mix']*enex.m32L/enex.s2m), legends = ['Service hot water', 'Hot water tank', 'Supply water to mixer'], time=config['time'], xlabel='Elapsed time [h]', ylabel='Flow rate [L/min]', xmax =xmax, savepath=f'{folder_path}/water_flows_{xmax}')

# plot_simple_graph(df['T_tank_w'], config['time'],'Elapsed time [h]', 'Tank water temp. [°C]', xmin = 0, xmax = xmax, color='dm.blue5', savepath = f'{folder_path}/T_tank_w_{xmax}')
plot_multi_graph((df['Q_use_loss'], df['Q_env_loss'], - df['Q_ref_cond']), legends = ['Heat loss by water use', 'Heat loss by envelope' , 'Heat input by GSHP'], time=config['time'], xlabel='Elapsed time [h]', ylabel='Heat rate [W]', xmax=xmax, savepath=f'{folder_path}/Q_losses_and_input_{xmax}')
# plot_simple_graph(df['T_b'], config['time'],'Elapsed time [h]', 'Borehole wall temp. [°C]', xmax = xmax, color='dm.violet5', Kelvin=True, savepath = f'{folder_path}/T_b_{xmax}')
# plot_multi_graph((df['T_b_f_in'], df['T_b_f_out']), legends = ['Borehole inlet water', 'Borehole outlet water'], time=config['time'], xlabel='Elapsed time [h]', ylabel='Temperature [°C]', xmax=xmax, Kelvin=True, savepath=f'{folder_path}/T_b_f_in_out_{xmax}')
plot_simple_graph(df['E_cmp'], config['time'],'Elapsed time [h]', 'Compressor power input [W]', xmax = xmax, color='dm.yellow7', savepath = f'{folder_path}/E_cmp_{xmax}')
# plot_simple_graph(df['COP'], config['time'],'Elapsed time [h]', 'COP [ - ]', xmax = xmax, color='dm.violet5', savepath = f'{folder_path}/COP_{xmax}')
# plot_multi_graph((df['Xg'], df['Xb'],), legends = ['Exergy from ground', 'Exergy from borehole'], time=config['time'], xlabel='Elapsed time [h]', ylabel='Exergy [W]', xmax=xmax, savepath=f'{folder_path}/Xg_Xb_{xmax}')
plot_simple_graph(df['Xc_g'], config['time'],'Elapsed time [h]', 'Ground exergy consum rate [W]', xmax = xmax, color='dm.orange5', savepath = f'{folder_path}/Xc_g_{xmax}')

In [41]:

# ===================== 사용자 입력(수정 지점) =====================
# TODO: 실제 데이터 경로/컬럼명으로 교체하세요.
CSV_PATH = 'gshpb_simulation_results_refactored.csv'  # 또는 당신의 CSV 경로
df = pd.read_csv(CSV_PATH)

X = config['time'] * enex.s2h  # 시간 [h]
Y1 = df['T_tank_w']
Y2 = df['dV_w_serv']

# 온수 사용량: 아래 둘 중 하나를 채워 쓰세요.   # [m^3/s]  (있으면 이걸 쓰고 L/min로 변환)
COL_LPM       = None          # [L/min]  (이미 L/min이면 이걸 지정, 위 COL_DV_M3s는 None)

TIME_UNIT = 's'  # 's'면 초→시간 변환, 'h'면 그대로 시간축 사용

# 플롯 범위/눈금(원하면 조정)
X_LABEL = 'Hour of day [h]'
Y1_LABEL = 'Tank water temperature [°C]'
Y2_LABEL = 'Hot water usage [L/min]'

xmin1, xmax1, xint1, xmar1 = 0, 24, 4, 0
ymin1, ymax1, yint1, ymar1 = 40, 70, 5, 0
ymin1_tw, ymax1_tw, yint1_tw = 0, 6, 1
color_ax1 = 'dm.blue'
color_ax1_twin = 'dm.red'
# ===============================================================


# 2) 데이터 로드

fig, ax1 = plt.subplots(1,1, figsize=(dm.cm2in(7), dm.cm2in(6)))

ax1.plot(X, Y1, label='', linewidth= LW[3], color=color_ax1 + '4')

# 5) plot detail
ax1.set_xlabel(X_LABEL, fontsize=fs['label'], labelpad=pad['label'])
ax1.set_ylabel('Temperature [°C]', fontsize=fs['label'], labelpad=pad['label'], color=color_ax1 + '6')


ax1.set_xlim(xmin1 - xmar1, xmax1 + xmar1)
ax1.set_ylim(ymin1 - ymar1, ymax1 + ymar1)

# ax1.annotate(annotations, xy=(.01, 1.01), xycoords='axes fraction',
#     horizontalalignment='left', verticalalignment='bottom', fontsize=fs['annotation']) 

ax1.set_xticks(np.arange(xmin1, xmax1*1.001, xint1))
ax1.set_yticks(np.arange(ymin1, ymax1*1.001, yint1))

ax1.spines['left'].set_visible(True)
ax1.spines['left'].set_color(color_ax1 + '6')

ax1.tick_params(labelsize=fs['tick'], which='major', length=2.5, width=0.3  , pad=pad['tick'])
ax1.tick_params(labelsize=fs['tick'], which='minor', length=1.25, width=0.3, pad=pad['tick'])

ax1.xaxis.set_minor_locator(ticker.AutoMinorLocator(2))
ax1.yaxis.set_minor_locator(ticker.AutoMinorLocator(2))

ax1.tick_params(axis='y', colors=color_ax1 + '6', labelsize=fs['tick'], which='major', pad=pad['tick'],)
ax1.tick_params(axis='y', colors=color_ax1 + '6', labelsize=fs['tick'], which='minor', pad=pad['tick'],)

handles, labels = ax1.get_legend_handles_labels()
ax1.legend(handles, labels, loc = 'upper center',fontsize=fs['legend'], bbox_to_anchor=(0.5, 1.1), ncols=5,
          handlelength = 1.5, columnspacing=2, labelspacing=0.5)

# 그래프 설정
ax1.set_xlabel(X_LABEL, fontsize=fs['label'], labelpad=pad['label'])
ax1.set_ylabel('Tank water temperature [°C]', fontsize=fs['label'], labelpad=pad['label'])

# 온수 사용 시점 시각적으로 표시
ax1_twin = ax1.twinx()
# ax1_twin.plot(X, Y2 * enex.m32L / enex.s2m, color=color_ax1_twin + '4', label='Hot water use [L/min]', linestyle='-.', linewidth=LW[2])
ax1_twin.fill_between(X, 0, Y2 * enex.m32L / enex.s2m, color=color_ax1_twin + '4', alpha=0.3, label='Hot water use [L/min]')
ax1_twin.set_ylabel('Hot water usage [L/min]', fontsize=fs['label'], labelpad=pad['label'], color=color_ax1_twin + '6')

ax1_twin.set_ylim(0, ymax1_tw)

ax1_twin.set_yticks(np.arange(ymin1_tw, ymax1_tw*1.001, yint1_tw))

for k in ['left', 'top', 'bottom']:
    ax1_twin.spines[k].set_visible(False)
ax1_twin.spines['right'].set_visible(True)
ax1_twin.spines['right'].set_color(color_ax1_twin + '6')
ax1_twin.tick_params(axis='y', colors=color_ax1_twin + '6', labelsize=fs['tick'], which='major', pad=pad['tick'],)
ax1_twin.tick_params(axis='y', colors=color_ax1_twin + '6', labelsize=fs['tick'], which='minor', pad=pad['tick'],)
ax1_twin.yaxis.set_minor_locator(ticker.AutoMinorLocator(2))

dm.simple_layout(fig, bbox=[0, 1, 0, 1], margins=[0.05, 0.05, 0.05, 0.05])
plt.savefig('figure/HeatPump_model/Tank_water_temperature.png', dpi=300)
dm.save_and_show(fig)


In [24]:

# ===================== 사용자 입력(수정 지점) =====================
# TODO: 실제 데이터 경로/컬럼명으로 교체하세요.
CSV_PATH = 'gshpb_simulation_results_refactored.csv'  # 또는 당신의 CSV 경로
df = pd.read_csv(CSV_PATH)

X = config['time'] * enex.s2h  # 시간 [h]
Y1 = enex.K2C(df['T_b'])
Y2 = df['Xc_g']

# 온수 사용량: 아래 둘 중 하나를 채워 쓰세요.   # [m^3/s]  (있으면 이걸 쓰고 L/min로 변환)
COL_LPM       = None          # [L/min]  (이미 L/min이면 이걸 지정, 위 COL_DV_M3s는 None)

TIME_UNIT = 's'  # 's'면 초→시간 변환, 'h'면 그대로 시간축 사용

# 플롯 범위/눈금(원하면 조정)
X_LABEL = 'Elapsed time [h]'
Y1_LABEL = 'Borehole wall temperature [°C]'
Y2_LABEL = 'Ground exergy consum rate [W]'

xmin1, xmax1, xint1, xmar1 = 0, 240, 24, 0
ymin1, ymax1, yint1, ymar1 = 0, 20, 4, 0
ymin1_tw, ymax1_tw, yint1_tw = 0, 30, 5
color_ax1 = 'dm.lime'
color_ax1_twin = 'dm.violet'
# ===============================================================


# 2) 데이터 로드

fig, ax1 = plt.subplots(1,1, figsize=(dm.cm2in(12), dm.cm2in(6)))

ax1.plot(X, Y1, label='', linewidth= LW[2], color=color_ax1 + '4')

# 5) plot detail
ax1.set_xlabel(X_LABEL, fontsize=fs['label'], labelpad=pad['label'])
ax1.set_ylabel(Y1_LABEL, fontsize=fs['label'], labelpad=pad['label'], color=color_ax1 + '6')


ax1.set_xlim(xmin1 - xmar1, xmax1 + xmar1)
ax1.set_ylim(ymin1 - ymar1, ymax1 + ymar1)

# ax1.annotate(annotations, xy=(.01, 1.01), xycoords='axes fraction',
#     horizontalalignment='left', verticalalignment='bottom', fontsize=fs['annotation']) 

ax1.set_xticks(np.arange(xmin1, xmax1*1.001, xint1))
ax1.set_yticks(np.arange(ymin1, ymax1*1.001, yint1))

ax1.spines['left'].set_visible(True)
ax1.spines['left'].set_color(color_ax1 + '6')

ax1.tick_params(labelsize=fs['tick'], which='major', length=2.5, width=0.3  , pad=pad['tick'])
ax1.tick_params(labelsize=fs['tick'], which='minor', length=1.25, width=0.3, pad=pad['tick'])

ax1.xaxis.set_minor_locator(ticker.AutoMinorLocator(2))
ax1.yaxis.set_minor_locator(ticker.AutoMinorLocator(2))

ax1.tick_params(axis='y', colors=color_ax1 + '6', labelsize=fs['tick'], which='major', pad=pad['tick'],)
ax1.tick_params(axis='y', colors=color_ax1 + '6', labelsize=fs['tick'], which='minor', pad=pad['tick'],)

handles, labels = ax1.get_legend_handles_labels()
ax1.legend(handles, labels, loc = 'upper center',fontsize=fs['legend'], bbox_to_anchor=(0.5, 1.1), ncols=5,
          handlelength = 1.5, columnspacing=2, labelspacing=0.5)

# 그래프 설정
ax1.set_xlabel(X_LABEL, fontsize=fs['label'], labelpad=pad['label'])
ax1.set_ylabel(Y1_LABEL, fontsize=fs['label'], labelpad=pad['label'])

# 온수 사용 시점 시각적으로 표시
ax1_twin = ax1.twinx()
ax1_twin.plot(X, Y2, color=color_ax1_twin + '4', label='Hot water use [L/min]', linestyle='-', linewidth=LW[1])
# ax1_twin.fill_between(X, 0, Y2, color=color_ax1_twin + '4', alpha=0.3, label='Hot water use [L/min]')
ax1_twin.set_ylabel(Y2_LABEL, fontsize=fs['label'], labelpad=pad['label'], color=color_ax1_twin + '6')

ax1_twin.set_ylim(0, ymax1_tw)

ax1_twin.set_yticks(np.arange(ymin1_tw, ymax1_tw*1.001, yint1_tw))
for k in ['left', 'top', 'bottom']:
    ax1_twin.spines[k].set_visible(False)
ax1_twin.spines['right'].set_visible(True)
ax1_twin.spines['right'].set_color(color_ax1_twin + '6')
ax1_twin.tick_params(axis='y', colors=color_ax1_twin + '6', labelsize=fs['tick'], which='major', pad=pad['tick'],)
ax1_twin.tick_params(axis='y', colors=color_ax1_twin + '6', labelsize=fs['tick'], which='minor', pad=pad['tick'],)
ax1_twin.yaxis.set_minor_locator(ticker.AutoMinorLocator(2))

dm.simple_layout(fig, bbox=[0, 1, 0, 1], margins=[0.05, 0.05, 0.05, 0.05])
plt.savefig('figure/HeatPump_model/ground_exergy_consumption_rate_240.png', dpi=300)
dm.save_and_show(fig)
